In [1]:
import numpy as np
import matplotlib.pyplot as plt
from CADMium import Pssolver, Psgrid, Partition, Inverter
import CADMium

from copy import copy

In [7]:
# dis_eq      = np.linspace(1.0,5,30)
# dis_st      = np.linspace(5.1,10,10)
# dis_eq      = np.linspace(1.0,5,10)
# dis_st      = np.linspace(5.1,10,3)
# distances   = np.concatenate((dis_eq, dis_st))
distances = [2.62] 
# distances = [2.0]
energy  = []

for d in distances:
    a = d/2
    Za, Zb =  9,9
    pol = 2

    #Set up grid
    NP = 7
    NM = [4,4]
    L = np.arccosh(15/a)
    loc = np.array(range(-4,5)) #Stencil outline
    grid = Psgrid(NP, NM, a, L, loc)
    grid.initialize()


    #Fragment a electrons [alpha, beta]

    #Fragment a electrons [alpha, beta]
    Nmo_a = [[2,2],[3,2]]; Nmo_A = [[2,2],[2,3]] #Number of molecular orbitals to calculate
    N_a   = [[2,2],[3,2]]; N_A   = [[2,2],[2,3]]
    nu_a = 0.5

    #Fragment b electrons
    Nmo_b = [[2,2],[3,2]]; Nmo_B = [[2,2],[2,3]]
    N_b   = [[2,2],[3,2]]; N_B   = [[2,2],[2,3]]
    nu_b = 0.5

    #Molecular elctron configuration
    Nmo_m = [[5,5],[4,4]]
    N_m   = [[5,5],[4,4]]



    part = Partition(grid, Za, Zb, pol, [Nmo_a, Nmo_A], [N_a, N_A], nu_a, 
                                        [Nmo_b, Nmo_B], [N_b, N_B], nu_b, {    "AB_SYM"            : True,
                                                                               "interaction_type"  : "dft", 
                                                                               "kinetic_part_type" : "inversion",
                                                                               "hxc_part_type"     : "exact",
#                                                                                "k_family"          : "gga", 
#                                                                                "ke_func_id"        : 500,
                                                                                })

    #Setup inverter object
    mol_solver = Pssolver(grid, Nmo_m, N_m)
    part.inverter = Inverter(grid, mol_solver, {  "AB_SYM"         : True, 
                                                  "use_iterative"  : False,
                                                  "invert_type"    : "orbitalinvert",
                                                  "DISP"           : True,  
                                                })

    part.optPartition.isolated = True
    part.scf({"disp"  : True,
              "alpha" : [0.4],
              "e_tol" : 1e-5})
    
#     atom = copy(part.E.Ea)
    
#     part.KSa.solver[0,0].optSolver.iter_lin_solver = False
#     part.KSa.solver[0,1].optSolver.iter_lin_solver = False
    
#     part.KSA.solver[0,0].optSolver.iter_lin_solver = False
#     part.KSA.solver[0,1].optSolver.iter_lin_solver = False
    
#     part.KSb.solver[0,0].optSolver.iter_lin_solver = False
#     part.KSb.solver[0,1].optSolver.iter_lin_solver = False
    
#     part.KSB.solver[0,0].optSolver.iter_lin_solver = False
#     part.KSB.solver[0,1].optSolver.iter_lin_solver = False

    part.optPartition.isolated   = False
    part.scf({"disp"       : True,
              "alpha"      : [0.6],
              "max_iter"   : 10,
              "e_tol"      : 5e-5,
              "iterative"  : False,
              "continuing" : True})

    energy.append(part.E.E)
    print(f"Done with {d}")
    
    
# energy    = np.array(energy)
# np.save('h2plus_distance.npy', distances)
# np.save('h2plus_overlap.npy', energy)

----> Begin SCF calculation for *Isolated* Fragments

                Total Energy (a.u.)                                Inversion                

                __________________                ____________________________________     

Iteration         A              B                  iters      optimality        res       

___________________________________________________________________________________________ 

    1         -131.56644   -131.56644       1.000e+00 
    2         -109.00854   -109.00854       2.411e-01 
    3         -102.78699   -102.78699       1.185e-01 
    4         -100.25107   -100.25107       6.690e-02 
    5          -99.15010    -99.15010       3.822e-02 
    6          -98.64045    -98.64045       2.193e-02 
    7          -98.39179    -98.39179       1.263e-02 
    8          -98.26638    -98.26638       7.291e-03 
    9          -98.21315    -98.21315       4.249e-03 
   10          -98.16883    -98.16883       2.432e-03 
   11          -98.151

TypeError: 'NoneType' object is not subscriptable

In [6]:
part.KSA.solver.shape

(2, 2)